# Calculate CMIP Indeces

This file 
1. calculates all the indeces for all months for all cesmFullForcingFiles.



In [1]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index
import utils.sstIndex as sst
import utils.pslIndex as psl
import utils.timePeriod as tp
import utils.compound as compound


In [2]:
import xarray
import numpy
import matplotlib.pyplot as plt

In [3]:
import warnings
#warnings.filterwarnings('ignore')

# 1. Calculate CMIP Indeces

For each experiment in historical, calculate indices

In [4]:
deckSet=['piControl','historical']
scenarioSet=['ssp126', 'ssp245', 'ssp370','ssp585']
experimentSet=[*deckSet, *scenarioSet]

In [6]:
modelSet=_model.scenarioMip[[2,12,14,18,42],:]

In [7]:
modelSet

array([['AWI', 'AWI-CM-1-1-MR', 'r1i1p1f1', 'r1i1p1f1'],
       ['CNRM-CERFACS', 'CNRM-CM6-1-HR', 'r1i1p1f2', 'r1i1p1f2'],
       ['CCCma', 'CanESM5', 'r1i1p1f1', 'r1i1p1f1'],
       ['EC-Earth-Consortium', 'EC-Earth3-CC', 'r1i1p1f1', 'r1i1p1f1'],
       ['MOHC', 'UKESM1-0-LL', 'r1i1p1f2', 'r1i1p1f2']], dtype='<U19')

In [8]:
sstIndeces = _index.sstIndex.keys()
pslIndeces = _index.pslIndex

In [9]:
len(modelSet)

5

Climatology and piControl:

In [ ]:
for iModel in modelSet:
    
    print(iModel)
    
    try: 
        #calculate climatology
        
        print(iModel[1] + ' starting')
        #SST
        tsDs = fh.loadModelData(iModel[1], 'ts_Amon', 'piControl', iModel[2]).ts.to_dataset()
        controlDs=tsDs.assign_attrs({'project_id':'CMIP'})
        sstClimat=sst.calculateClimatology(controlDs)

        [sstClimat[i].to_netcdf('results/cmipMonthlyIndeces/sstClimat'+iModel[1]+i+'.nc')
             for i in sstIndeces]

        pslControlDs=fh.loadModelData(iModel[1], 'psl_Amon', 'piControl', iModel[2])
        pslClimat=psl.calculateClimatology(pslControlDs)

        pslClimat.to_netcdf('results/cmipMonthlyIndeces/pslClimat'+iModel[1]+'.nc')

    except Exception as e:
        print(iModel[1] + "Climatology did not calculate")
        print(e)
 

['AWI' 'AWI-CM-1-1-MR' 'r1i1p1f1' 'r1i1p1f1']
AWI-CM-1-1-MR starting


/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1387: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


CMIP6.CMIP.AWI.AWI-CM-1-1-MR.piControl.r1i1p1f1.Amon.psl.gn.v20181218|esgf3.dkrz.de downloading


In [ ]:
for iModel in modelSet:
    
    try:
        sstClimat=dict()

        for i in sstIndeces:
            sstClimat[i]=xarray.open_dataarray('results/cmipMonthlyIndeces/sstClimat'+iModel[1]+i+'.nc')
        pslClimat=xarray.open_dataset('results/cmipMonthlyIndeces/pslClimat'+iModel[1]+'.nc')
        
        #the piControl
        sstIndex = sst.calculateIndex(controlDs, sstClimat) 
        pslIndex, junk = psl.calculateSamIndex(pslControlDs, pslClimat)
        indeces = xarray.merge([pslIndex, sstIndex])
        indeces.assign_attrs(climatology='full length of pi Control')
        print('Caclulating control ...')
        tp.averageForTimePeriod(indeces).to_netcdf(
            'results/cmipWarmSeasonIndeces/'+iModel[1]+'_piControl.nc')
        
    except Exception as e:
        print(iModel[1] + "piControl did not calculate")
        print(e)

Historical Indeces

In [ ]:
for iModel in modelSet:
    
    print(iModel)
    
    #try:    

    sstClimat=dict()

    for i in sstIndeces:
        sstClimat[i]=xarray.open_dataarray('results/cmipMonthlyIndeces/sstClimat'+iModel[1]+i+'.nc')
    pslClimat=xarray.open_dataset('results/cmipMonthlyIndeces/pslClimat'+iModel[1]+'.nc')

    #historical
    tsDs = fh.loadModelData(iModel[1], 'ts_Amon', 'historical', iModel[3]).ts.to_dataset()
    #sstDs = xarray.merge([tsDs.ts, fxDs.areacella])
    sstDs=tsDs.assign_attrs({'project_id':'CMIP'})

    sstIndex = sst.calculateIndex(sstDs, sstClimat) #(reducing the model set at this step could save time?)

    pslDs = fh.loadModelData(iModel[1], 'psl_Amon', 'historical',iModel[3])
    pslIndex, junk = psl.calculateSamIndex(pslDs, pslClimat)
    indeces = xarray.merge([pslIndex, sstIndex])
    indeces.assign_attrs(climatology='full length of pi Control')
    #print(indeces)
    print('Caclulating historical ...')

    #save the results to file
    indeces.to_netcdf('results/cmipMonthlyIndeces/'+iModel[1]+'_historical.nc')
    tp.averageForTimePeriod(indeces).to_netcdf('results/cmipWarmSeasonIndeces/'+iModel[1]+'_historical.nc')

    #except Exception as e:
    #    print(iModel[1] + "historical did not calculate")
    #    print(e)
        

Scenario Indeces

In [ ]:
for iModel in modelSet:
    
    print(iModel)
    
    try: 

#ref to the saved files
        historicalIndeces = xarray.open_dataset('results/cmipMonthlyIndeces/'+iModel[1]+'_historical.nc')
        
        sstClimat=dict()
        
        for i in sstIndeces:
            sstClimat[i]=xarray.open_dataarray('results/cmipMonthlyIndeces/sstClimat'+iModel[1]+i+'.nc')
        pslClimat=xarray.open_dataset('results/cmipMonthlyIndeces/pslClimat'+iModel[1]+'.nc')
        
        for experiment in scenarioSet: 
            try:
                variant = iModel[3]
                
                tsDs = fh.loadModelData(iModel[1], 'ts_Amon', experiment, variant).ts.to_dataset()
                #sstDs = xarray.merge([tsDs.ts, fxDs.areacella])
                sstDs=tsDs.assign_attrs({'project_id':'CMIP'})

                sstIndex = sst.calculateIndex(sstDs, sstClimat) #(reducing the model set at this step could save time?)

                pslDs = fh.loadModelData(iModel[1], 'psl_Amon', experiment,variant)
                pslIndex, junk = psl.calculateSamIndex(pslDs, pslClimat)

                indeces = xarray.concat([
                    historicalIndeces, 
                                         xarray.merge([pslIndex, sstIndex])
                    ], 'time')

                indeces.assign_attrs(climatology='full length of pi Control')
                #print(indeces)
                print('Caclulating ...')
                tp.averageForTimePeriod(indeces).to_netcdf('results/cmipWarmSeasonIndeces/'+iModel[1]+'_'+experiment + '.nc')

            except Exception as e:
                print(iModel[1] + experiment + " not completed: ")
                print(e)
                
            else:
                print(iModel[1] + experiment + ' complete')  

    except Exception as e:
        print(iModel[1] + "Climatology did not calculate")
        print(e)
        
    else:
        print(iModel[1] + ' finished')

This grabs the monthly CESM indeces, and calculates the averages for the time periods of interest (aka the warm season), as set in the _indexDefitionions file

Theres a few examples of output at the bottom to sanity check this is meaningful

The indeces for each month have already been calculated. So loading up the saved versions, and putting them into a single xarray.

In [ ]:
for experiment in experimentSet: 
    print(experiment)
    timePIndeces=list()

    for model in _model.scenarioMip[:,1]:
        try:
            indecesDs = xarray.open_dataset('results/cmipWarmSeasonIndeces/' + model +'_'+ experiment + '.nc')
            indecesDs['model']=model
            timePIndeces.append(indecesDs)
        except Exception as e:
            print(e) 
    results=xarray.merge(timePIndeces)

    results=results.assign_attrs({'experiment':experiment,**indecesDs.attrs,**_index.monthsOfInterest})

    results.to_netcdf('results/cmip6'+experiment+'TradIndeces.nc')

In [ ]:
results.sel(year=slice(1850,2099))

Some sanity checks to see if it looks ok:


In [ ]:
results=xarray.open_dataset('results/cmip6ssp585TradIndeces.nc').sel(year=slice(1850,2099))

In [ ]:
str(results.isel(model=6).model.values)

In [ ]:
# plotSomething
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1)

results.isel(model=5).nino34.plot(label='nino34')
results.isel(model=5).nino34NoDetrend.plot(label='No Detrend')

#Tidy up a bit
plt.title(str(results.isel(model=5).model.values) + " Historical Nino34")
plt.ylim(-4,4)
plt.minorticks_on()
plt.legend()

In [ ]:
# A multi-model mean?

mean = results.sam.mean(dim='model')
std = results.sam.std(dim='model')

In [ ]:
plt.figure()
mean.plot()
std.plot()

In [ ]:
# why limit yourself to one index?

mean = results.mean(dim='model')
std = results.std(dim='model')

In [ ]:
std

# 2. Calculate Compound Years for CESM6

# Define a criteria to determine events

So we are going to define events as follows:
- For SST indeces (aka Enso and Iod), this is 1 sigma above/below mean
- For PSL indeces (aka Sam) this is 0.5 sigma above/below mean

Using a piControl climatology

In [ ]:
piControlXr=xarray.open_dataset('results/cmip6piControlTradIndeces.nc')

piControlXr.model.values

In [ ]:
# This should calculate stds for all experiments and all indeces
stdXr = piControlXr.std(dim='year')

# define some criteria that we want to use to identifiy it thats an event
criteriaXr= xarray.merge(
    (stdXr[['nino34','dmi']], stdXr['sam']/2)
)

In [ ]:
criteriaXr

# Calculate the compounding years and plot an example

In [ ]:
for experiment in scenarioSet:

    # Here is the data
    indecesXr = xarray.open_dataset('results/cmip6'+experiment+'TradIndeces.nc')

    firePos, fireNeg = compound.applyCriteria(indecesXr, criteriaXr)

    firePos=compound.compound(firePos)

    fireNeg=compound.compound(fireNeg)

    firePos.to_netcdf('results/cmip6'+experiment+'FirePos.nc')

    fireNeg.to_netcdf('results/cmip6'+experiment+'FireNeg.nc')

In [ ]:
indecesXr.sel(year=2014)

In [ ]:
firePos

In [ ]:
modToPlot = 'CESM2'

toPlot=3*firePos.all3.sel(model=modToPlot)

plt.figure(figsize=(12,12))

plt.vlines(
    x=firePos.year, 
    ymin=0,
    ymax=toPlot,
    label='all3'
)

colors=['orange', 'purple', 'green']
iCol = 0

for iPair in firePos.attrs['pairs']:
    plt.vlines(
        x=firePos.year, 
        ymin=0,
        ymax=2*firePos[iPair].sel(model=modToPlot),
        color=colors[iCol],
        label=iPair
    )
    iCol+=1
plt.legend()
plt.xlim(1850,2100)
plt.title(modToPlot+': Pairs and all-three compounds for individal years')